## Importing Libraries

In [ ]:
import tensorflow as tf
from keras import layers
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
import numpy as np

In [ ]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

In [ ]:
import cv2
from ultralytics import YOLO
import os

In [ ]:

# Set the path to your dataset
train_dir = '/kaggle/input/deepfake-and-real-images/Dataset/Train'
validation_dir = '/kaggle/input/deepfake-and-real-images/Dataset/Validation'
test_dir = '/kaggle/input/deepfake-and-real-images/Dataset/Test'


## Preprocessing the images

### CS LBP

In [ ]:
# Define the function to calculate CS-LBP for a given pixel
def get_pixel(img, x1, y1, x, y):
    new_value = 0
    try:
        if img[x1][y1] >= img[x][y]:
            new_value = 1
    except IndexError:
        pass
    return new_value

def cs_lbp_calculated_pixel(img, x, y):
    val_ar = []
    
    val_ar.append(get_pixel(img, x, y+1, x, y-1))
    val_ar.append(get_pixel(img, x+1, y+1, x-1, y - 1))
    val_ar.append(get_pixel(img, x+1, y, x-1, y))
    val_ar.append(get_pixel(img, x+1, y-1, x - 1, y + 1))

    power_val = [1, 2, 4, 8]
    val = 0
    for i in range(len(val_ar)):
        val += val_ar[i] * power_val[i]
    return val

### CLAHE

In [ ]:
# Define the function to apply CS-LBP and CLAHE to an image
def apply_cs_lbp_clahe(image, output_directory, label, clip_limit=2.0, grid_size=(8, 8)):
    img_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    img_clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=grid_size).apply(img_gray)
    height, width = img_clahe.shape
    img_cs_lbp = np.zeros((height, width), np.uint16)

    for i in range(0, height):
        for j in range(0, width):
            img_cs_lbp[i, j] = cs_lbp_calculated_pixel(img_clahe, i, j)

    # Save the CS-LBP image
    cv2.imwrite(os.path.join(output_directory, label), img_cs_lbp)
    
    #plt.imshow(img_cs_lbp, cmap="gray")
    #plt.title("CS-LBP Image")
    #plt.show()


### YOLO

In [ ]:
# Load YOLO model
model = YOLO("/kaggle/input/modelbestx/best.pt")
    
# Function to extract face ROI from an image using YOLO
def extract_face(img_path):
    # Perform object detection
    results = model(img_path)
    boxes = results[0].boxes
    img = cv2.imread(img_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    if len(boxes) > 0:
        # Crop and save faces, and display them
        for i, box in enumerate(boxes):
            top_left_x = int(box.xyxy.tolist()[0][0])
            top_left_y = int(box.xyxy.tolist()[0][1])
            bottom_right_x = int(box.xyxy.tolist()[0][2])
            bottom_right_y = int(box.xyxy.tolist()[0][3])

            # Crop face region
            roi = img[top_left_y:bottom_right_y, top_left_x:bottom_right_x]
            return roi
    else:
        return None

clip_limit = 2.0
grid_size = (8, 8)

### Applying YOLO, CLAHE, CS LBP

In [ ]:
# Define the function to process each dataset
def process_dataset(real_dir, fake_dir, output_parent_dir):
    for label, directory in zip(['Real', 'Fake'], [real_dir, fake_dir]):
        output_dir = os.path.join(output_parent_dir, label)
        os.makedirs(output_dir, exist_ok=True)
        
        # Process each image in the directory
        for image_file in os.listdir(directory):
            image_path = os.path.join(directory, image_file)
            face_roi = extract_face(image_path)
            if face_roi is not None:
                apply_cs_lbp_clahe(face_roi, output_dir, image_file)

# Set the paths for each dataset
train_real_dir = '/kaggle/input/deepfake-and-real-images/Dataset/Train/Real'
train_fake_dir = '/kaggle/input/deepfake-and-real-images/Dataset/Train/Fake'
test_real_dir = '/kaggle/input/deepfake-and-real-images/Dataset/Test/Real'
test_fake_dir = '/kaggle/input/deepfake-and-real-images/Dataset/Test/Fake'
validation_real_dir = '/kaggle/input/deepfake-and-real-images/Dataset/Validation/Real'
validation_fake_dir = '/kaggle/input/deepfake-and-real-images/Dataset/Validation/Fake'
output_dir = '/kaggle/working/cslbp_images_clahe'

# Process each dataset
process_dataset(train_real_dir, train_fake_dir, os.path.join(output_dir, 'training'))
process_dataset(test_real_dir, test_fake_dir, os.path.join(output_dir, 'testing'))
process_dataset(validation_real_dir, validation_fake_dir, os.path.join(output_dir, 'validation'))

print("CS-LBP transformation with CLAHE completed for all images in the train, validation, and test datasets.")

In [ ]:
# Set the path to your dataset
train_dir = '/kaggle/working/cslbp_images_clahe/training'
validation_dir = '/kaggle/working/cslbp_images_clahe/validation'
test_dir = '/kaggle/working/cslbp_images_clahe/testing'

### Data Augmentation and Normalization

In [ ]:
# Define data augmentation and normalization
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)


### Resizing the Images

In [ ]:
# Use datagen.flow_from_directory() to load and preprocess your dataset
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary',  # Assuming a binary classification task (fake vs real)
    classes=['Fake','Real']  # Specify the class names
    
)


In [ ]:
validation_generator = datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary',
    classes=['Fake', 'Real']
)

In [ ]:
test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary',
    classes=['Fake', 'Real']
)

## Training the model

### Compiling the CNN

In [ ]:
# Define the model
model = Sequential()

# Convolutional Layer 1
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(224, 224, 3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.25))

# Convolutional Layer 2
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Convolutional Layer 3
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Fully Connected Layers
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Display the model summary
model.summary()

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

### Training the CNN

In [ ]:
# Train the model
model.fit(train_generator, epochs=40, validation_data=validation_generator)


In [ ]:
# Create the output directory if it doesn't exist
output_dir = '/kaggle/output/'
os.makedirs(output_dir, exist_ok=True)

# Save the model to the output directory
model.save(os.path.join(output_dir, 'final_model.keras'))


In [ ]:
from keras.models import load_model

# Replace 'final_model.h5' with the actual file name you used to save your model
saved_model_path = '/kaggle/output/final_model.keras'

# Load the saved model
loaded_model = load_model(saved_model_path)

# Now you can use 'loaded_model' for inference or further training


### Testing the Performance of the Model

In [ ]:
from keras.preprocessing import image
import numpy as np

# Define the path to the new image you want to classify
new_image_path = '/kaggle/working/extracted_faces/Test/Fake/fake_3300.jpg'  

# Load the image and preprocess it for the model
img = image.load_img(new_image_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.  # Normalize the pixel values

# Make predictions using the loaded model
predictions = loaded_model.predict(img_array)

# Assuming binary classification (real vs. fake)
# You may have to adjust this based on your model's output
if predictions[0, 0] > 0.5:
    print("Prediction: Real Image")
else:
    print("Prediction: Fake Image")


In [ ]:
# Evaluate the model on the test_generator data
evaluation = loaded_model.evaluate(test_generator)

# The evaluate method returns the loss and accuracy
loss, accuracy = evaluation

print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy * 100:.2f}%')